In [1]:
from fitting import get_model, get_model_name, evaluate
from dataprocessing import generate_tasks
from meta import get_class_weights, Meta
import numpy as np
import torch

# Load the model

In [4]:
# Model architecture
hidden_layers = [85, 91]

# Arguments to create model
sup_shots = 100
que_shots = 200
dropout = 0.07
name = get_model_name(sup_shots, que_shots, dropout, hidden_layers)
config = get_model(hidden_layers, dropout)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Model filename
filename = "models/" + name + ".pt"

# Choose PyTorch device and load the model
model = Meta(name, config, sup_shots, que_shots, device).to(device)
model.load(filename)

# Load the data

In [5]:
# Datapath and background file for data-files
datapath = "processed-data/"
bkg_file = datapath + "bkg.h5"

# Signal files for each task split
val_signals = ["hg3000_hq1200", "wohg_hq1000"]
test_signals = ["wohg_hq1400", "fcnc"]

# Add datapath and extention to files for each split
val_signals = [datapath + p + ".h5" for p in val_signals]
test_signals = [datapath + p + ".h5" for p in test_signals]

# Generate tasks
sup_shots = 100
que_shots = 200
val_tasks = generate_tasks(val_signals, bkg_file, sup_shots, que_shots)
test_tasks = generate_tasks(test_signals, bkg_file, sup_shots, que_shots)

# Evaluate on the val and test data

In [6]:

# evaluate(model, val_tasks, 200, "Eval Val")

# Get support sample
x_sup, w_sup, y_sup = next(test_tasks["fcnc"]["sup"]["data"])
class_weights = get_class_weights(test_tasks["fcnc"]["sup"]["weights"], y_sup)

# Get query sample to predict on
x_que, w_que, y_que = next(test_tasks["fcnc"]["que"]["data"])
y_hat = model.predict(x_sup, w_sup, y_sup, x_que, class_weights)